<a href="https://colab.research.google.com/github/karenbennis/Xy/blob/Reformation/ml_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>**Connect to Database**<br><br>

In [2]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#Interact with SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark Session(Creating spark application with name defined by appName()) ---IMPORTED WITH EVERY COLAB NOTEBOOK
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("database_transformation").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


--2020-07-20 19:56:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.63MB/s    in 0.2s    

2020-07-20 19:56:15 (3.63 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [3]:
# Authenticate user
from google.colab import auth
auth.authenticate_user()

In [4]:
# Set project id
project_id = 'xy-yelp'

In [5]:
# Set project
!gcloud config set project {project_id}

Updated property [core/project].


In [6]:
# Get file which was saved to bucket **** file saves as multiple files so this should be addressed in the future, for now the 11mb file should be fine
!gsutil cp -r gs://xy-bucket/json_files/ml_j.json/part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json /tmp/machine_json.json

Copying gs://xy-bucket/json_files/ml_j.json/part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json...
-
Operation completed over 1 objects/62.5 MiB.                                     


In [7]:
# Set filepath to tmp folder where bucket file was saved **** this could actually be done without saving to the bucket, this is just done to make sure we can 
# save to and read from bucket if we need to
filepath = '/tmp/machine_json.json'

In [8]:
# Import pandas and read json file into DataFrame
import pandas as pd
raw_json = pd.read_json(filepath, lines=True)

In [9]:
#Drop useless columns
raw_json=raw_json[['review_id','stars','stars_one_hot','features']]

In [10]:
# Show dataframe
raw_json.head()

,review_id,stars,stars_one_hot,features
0,-7yxrdY13ay15rGB7WibMA,5,"{'type': 0, 'size': 5, 'indices': [1], 'values...","{'type': 0, 'size': 262145, 'indices': [9521, ..."
1,-Be0UUGYuiDJVAM_YqeQuA,4,"{'type': 0, 'size': 5, 'indices': [0], 'values...","{'type': 0, 'size': 262145, 'indices': [78, 31..."
2,-nQHHXi-d_yuW301_Y0EZQ,2,"{'type': 0, 'size': 5, 'indices': [3], 'values...","{'type': 0, 'size': 262145, 'indices': [1846, ..."
3,2L30O7G8IQ6HILpR0t5RFA,5,"{'type': 0, 'size': 5, 'indices': [1], 'values...","{'type': 0, 'size': 262145, 'indices': [8804, ..."
4,4x5yLG7_yGLuN-w6fV0eBw,4,"{'type': 0, 'size': 5, 'indices': [0], 'values...","{'type': 0, 'size': 262145, 'indices': [17141,..."


In [11]:
# View full contents of cell to understand how json data was read into DataFrame
y=raw_json.iat[0,-1]
print(y)

{'type': 0, 'size': 262145, 'indices': [9521, 13381, 24113, 34146, 47205, 48870, 52657, 55639, 69793, 72944, 74473, 78329, 79660, 80245, 82321, 82582, 87910, 89717, 95454, 95502, 101376, 107367, 109230, 113458, 113462, 115157, 117481, 118144, 120391, 125372, 128924, 130707, 132270, 132538, 134691, 137431, 138356, 141407, 154186, 157120, 168385, 181758, 184251, 188828, 192137, 195155, 208258, 216432, 218117, 221790, 222394, 229772, 232685, 235700, 236821, 240976, 245044, 262144], 'values': [13.146379889022644, 4.547001272864449, 1.6436118877681851, 12.331835858506187, 10.720585530532915, 4.213228093212401, 6.90785527398247, 2.310717259691643, 8.111828078308406, 7.824146005856625, 3.139702638974026, 2.464028238403142, 7.418680897748461, 4.625472888305944, 4.213228093212401, 13.004780331748613, 6.725533717188516, 2.067612965814895, 8.517293186416572, 8.517293186416572, 3.973998404146567, 2.415853786100017, 10.014967732503301, 2.852598100468416, 5.083305981931424, 3.5302677579594492, 3.249

In [12]:
# Separate dictionary values keys into columns keeping the values and column values
features_series = raw_json.features.apply(pd.Series)

In [13]:
# Drop type as it was an extra json value added during conversion
features_series = features_series.drop('type', axis=1)

In [14]:
# Check DataFrame structure
features_series.head(3)

,size,indices,values
0,262145,"[9521, 13381, 24113, 34146, 47205, 48870, 5265...","[13.146379889022644, 4.547001272864449, 1.6436..."
1,262145,"[78, 3188, 4200, 4821, 5381, 5947, 8391, 8527,...","[3.9846936932633152, 8.111828078308406, 2.9151..."
2,262145,"[1846, 4106, 7917, 8287, 8630, 8769, 13677, 15...","[11.89024888444809, 6.812545094178145, 4.77962..."


In [15]:
# Initialize target dataframe
new_df = features_series.copy()

In [21]:
# Combine size, indices, and values into tuple containing mutil-dimensional array to match original format
new_df['features'] = list(zip(features_series['indices'], features_series['values']))
new_df=new_df.rename(columns={"indices": "feature_indices", "values": "feature_values"})
new_df=new_df.drop('size',axis=1)

In [22]:
# Inspect DataFrame
new_df.head()

,feature_indices,feature_values,features
0,"[9521, 13381, 24113, 34146, 47205, 48870, 5265...","[13.146379889022644, 4.547001272864449, 1.6436...","([9521, 13381, 24113, 34146, 47205, 48870, 526..."
1,"[78, 3188, 4200, 4821, 5381, 5947, 8391, 8527,...","[3.9846936932633152, 8.111828078308406, 2.9151...","([78, 3188, 4200, 4821, 5381, 5947, 8391, 8527..."
2,"[1846, 4106, 7917, 8287, 8630, 8769, 13677, 15...","[11.89024888444809, 6.812545094178145, 4.77962...","([1846, 4106, 7917, 8287, 8630, 8769, 13677, 1..."
3,"[8804, 19862, 30006, 40337, 46639, 47032, 5099...","[6.593874722676491, 5.036053097080879, 2.57975...","([8804, 19862, 30006, 40337, 46639, 47032, 509..."
4,"[17141, 24145, 35715, 48549, 54961, 61231, 755...","[5.259196648395089, 3.550958151216895, 4.47424...","([17141, 24145, 35715, 48549, 54961, 61231, 75..."


In [23]:
# Start converting label data, again unzipping dictionary keys to columns and values as rows
labels_series = raw_json.stars_one_hot.apply(pd.Series)
labels_series.head()

,type,size,indices,values
0,0,5,[1],[1.0]
1,0,5,[0],[1.0]
2,0,5,[3],[1.0]
3,0,5,[1],[1.0]
4,0,5,[0],[1.0]


In [24]:
# Drope type since its not needed
labels_series = labels_series.drop('type', axis=1)
labels_series.head(3)

,size,indices,values
0,5,[1],[1.0]
1,5,[0],[1.0]
2,5,[3],[1.0]


In [25]:
# Initialize DataFrame
newer_df = labels_series.copy()

In [26]:
# Combine size, indices, and values into tuple containing mutil-dimensional array to match original format
newer_df['stars_ohe'] = list(zip(labels_series['size'], labels_series['indices'], labels_series['values']))
newer_df.head(3)

,size,indices,values,stars_ohe
0,5,[1],[1.0],"(5, [1], [1.0])"
1,5,[0],[1.0],"(5, [0], [1.0])"
2,5,[3],[1.0],"(5, [3], [1.0])"


In [27]:
# Drop redundant columns
newer_df = newer_df.drop(['size', 'indices', 'values'], axis=1)
newer_df.head(3)

,stars_ohe
0,"(5, [1], [1.0])"
1,"(5, [0], [1.0])"
2,"(5, [3], [1.0])"


In [28]:
# Combine features and labels to DataFrame to be used in machine learning model
machine_df = pd.concat([ raw_json[['review_id','stars']], new_df, newer_df], axis=1)
machine_df.head()

,review_id,stars,feature_indices,feature_values,features,stars_ohe
0,-7yxrdY13ay15rGB7WibMA,5,"[9521, 13381, 24113, 34146, 47205, 48870, 5265...","[13.146379889022644, 4.547001272864449, 1.6436...","([9521, 13381, 24113, 34146, 47205, 48870, 526...","(5, [1], [1.0])"
1,-Be0UUGYuiDJVAM_YqeQuA,4,"[78, 3188, 4200, 4821, 5381, 5947, 8391, 8527,...","[3.9846936932633152, 8.111828078308406, 2.9151...","([78, 3188, 4200, 4821, 5381, 5947, 8391, 8527...","(5, [0], [1.0])"
2,-nQHHXi-d_yuW301_Y0EZQ,2,"[1846, 4106, 7917, 8287, 8630, 8769, 13677, 15...","[11.89024888444809, 6.812545094178145, 4.77962...","([1846, 4106, 7917, 8287, 8630, 8769, 13677, 1...","(5, [3], [1.0])"
3,2L30O7G8IQ6HILpR0t5RFA,5,"[8804, 19862, 30006, 40337, 46639, 47032, 5099...","[6.593874722676491, 5.036053097080879, 2.57975...","([8804, 19862, 30006, 40337, 46639, 47032, 509...","(5, [1], [1.0])"
4,4x5yLG7_yGLuN-w6fV0eBw,4,"[17141, 24145, 35715, 48549, 54961, 61231, 755...","[5.259196648395089, 3.550958151216895, 4.47424...","([17141, 24145, 35715, 48549, 54961, 61231, 75...","(5, [0], [1.0])"


In [29]:
x=machine_df[['review_id','feature_indices','feature_values']]
x.head()

,review_id,feature_indices,feature_values
0,-7yxrdY13ay15rGB7WibMA,"[9521, 13381, 24113, 34146, 47205, 48870, 5265...","[13.146379889022644, 4.547001272864449, 1.6436..."
1,-Be0UUGYuiDJVAM_YqeQuA,"[78, 3188, 4200, 4821, 5381, 5947, 8391, 8527,...","[3.9846936932633152, 8.111828078308406, 2.9151..."
2,-nQHHXi-d_yuW301_Y0EZQ,"[1846, 4106, 7917, 8287, 8630, 8769, 13677, 15...","[11.89024888444809, 6.812545094178145, 4.77962..."
3,2L30O7G8IQ6HILpR0t5RFA,"[8804, 19862, 30006, 40337, 46639, 47032, 5099...","[6.593874722676491, 5.036053097080879, 2.57975..."
4,4x5yLG7_yGLuN-w6fV0eBw,"[17141, 24145, 35715, 48549, 54961, 61231, 755...","[5.259196648395089, 3.550958151216895, 4.47424..."


In [30]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [31]:
import numpy as np
y=explode(x, lst_cols=['feature_indices','feature_values'])
y.head()

,review_id,feature_indices,feature_values
0,-7yxrdY13ay15rGB7WibMA,9521,13.146380
1,-7yxrdY13ay15rGB7WibMA,13381,4.547001
2,-7yxrdY13ay15rGB7WibMA,24113,1.643612
3,-7yxrdY13ay15rGB7WibMA,34146,12.331836
4,-7yxrdY13ay15rGB7WibMA,47205,10.720586


In [32]:
y=y.set_index('review_id')
a=y.pivot(columns='feature_indices',values='feature_values')
a.head()

feature_indices,5,6,8,10,13,14,15,43,47,52,61,67,68,70,78,90,107,116,123,125,132,133,145,150,168,170,181,182,191,193,194,200,204,205,211,216,223,227,228,239,...,261974,261975,261983,261986,261987,261995,261996,262001,262014,262028,262029,262035,262040,262042,262047,262048,262051,262054,262055,262067,262071,262072,262075,262077,262084,262091,262096,262099,262101,262105,262110,262113,262117,262120,262122,262125,262136,262137,262142,262144
review_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--8hpKO4bkweLyEucdxI_w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527.0
--DazeDpOApabZnIOIPSrQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1131.0
--QCUEmDBlipC_CEutGVFA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,799.0
--TqfVGywiYWh7Sn9ksBmA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0
-04V-J248jjwibqilfLePA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

In [ ]:
x=machine_df['indices']
y=machine_df['stars']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=101)

In [110]:
list('')

['B', 'C', 'D', 'E']

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
# it doesnt like that x_train is a list

ValueError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(reviews['text'])

In [ ]:
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(vectors)